# Boost factors

In [ ]:
import clmm
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.galaxycluster import GalaxyCluster
import matplotlib.pyplot as plt
import sys
import clmm.utils as u

Make sure we know which version we're using

In [ ]:
clmm.__version__

### Define cosmology object

In [ ]:
mock_cosmo = Cosmology(H0=70.0, Omega_dm0=0.27 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

First, we want to generate a $\Delta\Sigma$ (excess surface density) profile from mock data, to which we can apply boost factors. The mock data is generated in the following cells.

Generate cluster object from mock data

In [ ]:
cosmo = mock_cosmo
cluster_id = "Awesome_cluster"
cluster_m = 1.0e15
cluster_z = 0.3
concentration = 4
ngals = 1000

zsrc_min = cluster_z + 0.1  # we only want to draw background galaxies

noisy_data_z = mock.generate_galaxy_catalog(
    cluster_m,
    cluster_z,
    concentration,
    cosmo,
    "chang13",
    zsrc_min=zsrc_min,
    shapenoise=0.005,
    photoz_sigma_unscaled=0.05,
    ngals=ngals,
)

Loading this into a CLMM cluster object centered on (0,0)

In [ ]:
cluster_ra = 0.0
cluster_dec = 0.0
cl = GalaxyCluster(cluster_id, cluster_ra, cluster_dec, cluster_z, noisy_data_z)

Compute cross and tangential excess surface density for each source galaxy

In [ ]:
_ = cl.compute_tangential_and_cross_components(
    geometry="flat",
    shape_component1="e1",
    shape_component2="e2",
    tan_component="DeltaSigma_tan",
    cross_component="DeltaSigma_cross",
    add=True,
    cosmo=cosmo,
    is_deltasigma=True,
)

Calculate the binned profile

In [ ]:
cl.make_radial_profile(
    "kpc",
    cosmo=cosmo,
    tan_component_in="DeltaSigma_tan",
    cross_component_in="DeltaSigma_cross",
    tan_component_out="DeltaSigma_tan",
    cross_component_out="DeltaSigma_cross",
    table_name="DeltaSigma_profile",
)
# Format columns for display
for col in cl.DeltaSigma_profile.colnames:
    fmt = cl.DeltaSigma_profile[col].info.format
    if "DeltaSigma" in col:
        fmt = ".2e"
    elif any(typ in col for typ in ("z", "radius")):
        fmt = ".2f"
    cl.DeltaSigma_profile[col].info.format = fmt
# Show
cl.DeltaSigma_profile.show_in_notebook()

Plot the $\Delta\Sigma$ profile

In [ ]:
plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    cl.DeltaSigma_profile["DeltaSigma_tan"],
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
)

plt.title("DeltaSigma profile")
plt.xlabel("Radius [kpc]")
plt.ylabel("$\Delta\Sigma [M_\odot\; Mpc^{-2}]$")

plt.show()

## Boost Factors

CLMM offers two boost models, the NFW boost model, and a powerlaw boost model. 

- `compute_nfw_boost` requires two parameters to be specified, `rs` (non-optional) and `b0` (optional)
- `compute_powerlaw_boost` requires three paramters, `rs`(non-optional), `b0`(optional) and `alpha`(optional). 

Details on these boost models can be found [here](https://cluster-toolkit.readthedocs.io/en/latest/source/boostfactors.html)

First, we can calculate the boost factors from the two models.

In [ ]:
r_proj = cl.DeltaSigma_profile["radius"]
r_scale = 1000 # in kpc to match r_proj units
nfw_boost = u.compute_nfw_boost(r_proj, r_scale, boost0=0.1)

powerlaw_boost = u.compute_powerlaw_boost(r_proj, r_scale, boost0=0.1, alpha=-1.0
)

Plot the two boost factors, $B(R)$

In [ ]:
plt.loglog(r_proj, nfw_boost, label="NFW boost factor", marker='.')
plt.loglog(r_proj, powerlaw_boost, label="Powerlaw boost factor", marker='.')
plt.xlabel("Radius [kpc]")
plt.ylabel("$B(R)$")
plt.legend()
plt.show()

The $\Delta\Sigma$ profiles can be corrected with the boost factor using `correct_sigma_with_boost_values` or `correct_sigma_with_boost_model`. 

`correct_sigma_with_boost_values` requires us to precompute the boost factor, e.g. using `compute_nfw_boost`.
`correct_sigma_with_boost_model` simply requires us to specify the boost model.


Note that the boost factor can be used in one of two ways.

Either the boost factor can be applied to the observed data vector to correct for the dilution of the signal by cluster member galaxies. In this case the amplitude of the corrected profile will increase.

Or the boost factor can be applied to the model prediction. In this case it behaves as a dilution factor, and the resulting model prediction will be lower than the original one.

Both scenarios will improve the agreement between the mock data and observed data, by accounting for cluster member galaxy contamination.

In this notebook, we use the second approach, where the data is generated using mock data that does not account for dilution until the boost factor is applied. The corrected profiles from the mock data are lower than the uncorrected one.

Essentially we are diluting the mock profile to mimick the effect of contamination by cluster members.



First we will apply the boost factor with `correct_sigma_with_boost_values`

In [ ]:
Sigma_corrected_powerlaw_boost = u.correct_with_boost_values(
    cl.DeltaSigma_profile["DeltaSigma_tan"], powerlaw_boost
)
Sigma_corrected_nfw_boost = u.correct_with_boost_values(
    cl.DeltaSigma_profile["DeltaSigma_tan"], nfw_boost
)

Plot the result

In [ ]:
plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    Sigma_corrected_nfw_boost,
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="$\Delta \Sigma$ / NFW boost factor",
)

plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    Sigma_corrected_powerlaw_boost,
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="$\Delta \Sigma$ / Powerlaw boost factor",
)

plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    cl.DeltaSigma_profile["DeltaSigma_tan"],
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="uncorrected $\Delta \Sigma$",
    color="grey",
)

#plt.loglog()
plt.title("DeltaSigma profile")
plt.xlabel("Radius [kpc]")
plt.ylabel("$\Delta\Sigma [M_\odot\; Mpc^{-2}]$")
plt.legend()
plt.show()

Now the same again but with `correct_with_boost_model`

In [ ]:
Sigma_corrected_powerlaw_boost = u.correct_with_boost_model(
    cl.DeltaSigma_profile["radius"],
    cl.DeltaSigma_profile["DeltaSigma_tan"],
    "powerlaw_boost", # boost_model
    r_scale, # boost_rscale (in units of cl.DeltaSigma_profile["radius"])
)
Sigma_corrected_nfw_boost = u.correct_with_boost_model(
    cl.DeltaSigma_profile["radius"],
    cl.DeltaSigma_profile["DeltaSigma_tan"],
    "nfw_boost", # boost_model
    r_scale, # boost_rscale (in units of cl.DeltaSigma_profile["radius"])
)

plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    Sigma_corrected_nfw_boost,
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="$\Delta \Sigma$ / NFW boost factor",
)

plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    Sigma_corrected_powerlaw_boost,
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="$\Delta \Sigma$ / Powerlaw boost factor",
)

plt.errorbar(
    cl.DeltaSigma_profile["radius"],
    cl.DeltaSigma_profile["DeltaSigma_tan"],
    cl.DeltaSigma_profile["DeltaSigma_tan_err"],
    marker="o",
    label="uncorrected $\Delta \Sigma$",
    color="grey",
)

# plt.loglog()
plt.title("DeltaSigma profile")
plt.xlabel("Radius [kpc]")
plt.ylabel("$\Delta\Sigma [M_\odot\; Mpc^{-2}]$")
plt.legend()
plt.show()